# Big Data - Project

## TODO TOPIC

### Dragan Postolovsky
### Wojciech Taisner

TODO

A 1-page summary of litterature review on studies investigating stackoverflow data

TODO

An enumeration of "interesting" investigations directions for the 10-year dataset of stack overflow (from 09/2008 to 09/2018)

### What were the questions?

TODO

A report with analysis results (including visualisation, and statistics) of the 10 years data. These results are the fruit of the investigations projected earlier

In [1]:
# import dask bag
import dask.bag as db
# import BeatuifulSoup
from bs4 import BeautifulSoup

In [2]:
#files
base = "stackoverflow.com-"

badges = "Badges"
comments = "Comments"
posthistory = "PostHistory"
postlinks = "PostLinks"
posts = "Posts"
tags = "Tags"
users = "Users"
votes = "Votes"

all_files = [badges, comments, postlinks, posts, tags, users, votes] # posthistory, 
# for some reason post history does not want to co-operate
all_files

['Badges', 'Comments', 'PostLinks', 'Posts', 'Tags', 'Users', 'Votes']

# Functions

In [3]:
# non complex parsing (ex badges, tags, etc)
def xml_row_to_tuple(row): # gets data from single xml row (attr values)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        if v.isdigit():
            v = int(v) #integers take less memory :D
        ret.append(v)
    return tuple(ret);


def xml_headers_to_tuple(row): # gets headers for single xml row (attr names)
    soup = BeautifulSoup(row, "xml")
    ret = []
    for a, v in soup.row.attrs.items():
        ret.append(a)
    return tuple(ret);

In [4]:
# TODO read bag function

In [5]:
# check if bag is ok (data are more-a-less structured)
# takes a while of course
def check_bag(bag):
    ma = bag.map(xml_row_to_tuple).map(lambda x: len(x)).max().compute()
    mi = bag.map(xml_row_to_tuple).map(lambda x: len(x)).min().compute()
    if ma == mi:
        return True
    print(ma, mi)
    return False

## Review of files structure (probably to be removed later)

In [6]:
# refiew structure of all files
# takes a while
extension=".tar.xz"
compression="xz"
for filename in all_files:
    print(filename)
    temp_bag = db.read_text(filename+extension, compression=compression).str.strip()
    temp_bag = temp_bag.filter(lambda x: x.find("row") >= 0)
    th = temp_bag.map(xml_headers_to_tuple).take(1)[0]
    #print(th)
    for i, v in enumerate(th):
        print('{}: {}'.format(i, v))
    #too long for showcase only
    #t = temp_bag.map(xml_row_to_tuple).compute()
    #print(t[0:4])
    #tp = list(temp_bag.map(xml_row_to_tuple).take(4))
    #for t in tp:
    #    print(t)
    tp = temp_bag.map(xml_row_to_tuple).take(1)[0]
    for i, v in enumerate(tp):
        print('{}: {}'.format(i, v))
    

Badges
0: Id
1: UserId
2: Name
3: Date
4: Class
5: TagBased
0: 82946
1: 3718
2: Teacher
3: 2008-09-15T08:55:03.923
4: 3
5: False
Comments
0: Id
1: PostId
2: Score
3: Text
4: CreationDate
5: UserId
0: 2
1: 35314
2: 8
3: Yeah, I didn't believe it until I created a console app - but good lord!  Why would they give you the rope to hang yourself!  I hated that about VB.NET - the OrElse and AndAlso keywords!
4: 2008-09-06T08:09:52.330
5: 3
PostLinks
0: Id
1: CreationDate
2: PostId
3: RelatedPostId
4: LinkTypeId
0: 19
1: 2010-04-26T02:59:48.130
2: 109
3: 32412
4: 1
Posts
0: Id
1: PostTypeId
2: AcceptedAnswerId
3: CreationDate
4: Score
5: ViewCount
6: Body
7: OwnerUserId
8: LastEditorUserId
9: LastEditorDisplayName
10: LastEditDate
11: LastActivityDate
12: Title
13: Tags
14: AnswerCount
15: CommentCount
16: FavoriteCount
0: 6
1: 1
2: 31
3: 2008-07-31T22:08:08.620
4: 261
5: 16799
6: <p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively pos

# Some real stuff

## Tags

In [7]:
# read tags bag, select rows only
tags_bag = db.read_text(tags+".tar.xz", compression="xz").str.strip()
tags_bag = tags_bag.filter(lambda x: x.find("row") >= 0)

### Simple tests [Polygon]

In [8]:
for row in tags_bag.take(4):
    print(row);
    print(xml_row_to_tuple(row))
    #soup = BeautifulSoup(row, "xml")
    #print(soup.row['Id'], soup.row['TagName'], soup.row['Count'])

<row Id="1" TagName=".net" Count="276857" ExcerptPostId="3624959" WikiPostId="3607476" />
(1, '.net', 276857, 3624959, 3607476)
<row Id="2" TagName="html" Count="789778" ExcerptPostId="3673183" WikiPostId="3673182" />
(2, 'html', 789778, 3673183, 3673182)
<row Id="3" TagName="javascript" Count="1723731" ExcerptPostId="3624960" WikiPostId="3607052" />
(3, 'javascript', 1723731, 3624960, 3607052)
<row Id="4" TagName="css" Count="563816" ExcerptPostId="3644670" WikiPostId="3644669" />
(4, 'css', 563816, 3644670, 3644669)


In [9]:
tags_bag.take(4) # return tuple
test = tags_bag.map(xml_row_to_tuple).compute()
test[0]
#carefull, takes a while

(1, '.net', 276857, 3624959, 3607476)

In [10]:
header = tags_bag.map(xml_headers_to_tuple).take(1)[0]
header

('Id', 'TagName', 'Count', 'ExcerptPostId', 'WikiPostId')

In [11]:
print(type(tags_bag))
print(type(test))
#tags_bag.map(xml_row_to_tuple).take(1)

<class 'dask.bag.core.Bag'>
<class 'list'>


In [12]:
# takes a while
check_bag(tags_bag)

5 3


False

In [13]:
tags_bag.map(xml_row_to_tuple).filter(lambda x: len(x)<5).count().compute()

14218

In [14]:
tags_bag.map(xml_headers_to_tuple).filter(lambda x: len(x)==3).distinct().compute()
#tags_bag.count().compute()
#
# 14k / 55k has only 3/5 tags (two last missing), need to be dropped
#

[('Id', 'TagName', 'Count')]

## Posts and Answers 

In [15]:
## split posts to posts and answers
# takes a while
# quite a long ...
# man im gonna take a shower
both_bag = db.read_text(posts+".tar.xz", compression="xz").str.strip()
both_bag = both_bag.filter(lambda x: x.find('row') >=0 )
#both_bag.count().compute()

In [16]:
answers_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"2\"") >= 0)
answers_bag.map(xml_row_to_tuple).take(1)
#answers_bag.count().compute()

((12,
  2,
  11,
  '2008-07-31T23:56:41.303',
  323,
  '<p>Here\'s how I do it</p>\n\n<pre class="lang-csharp prettyprint-override"><code>var ts = new TimeSpan(DateTime.UtcNow.Ticks - dt.Ticks);\ndouble delta = Math.Abs(ts.TotalSeconds);\n\nif (delta &lt; 60)\n{\n  return ts.Seconds == 1 ? "one second ago" : ts.Seconds + " seconds ago";\n}\nif (delta &lt; 120)\n{\n  return "a minute ago";\n}\nif (delta &lt; 2700) // 45 * 60\n{\n  return ts.Minutes + " minutes ago";\n}\nif (delta &lt; 5400) // 90 * 60\n{\n  return "an hour ago";\n}\nif (delta &lt; 86400) // 24 * 60 * 60\n{\n  return ts.Hours + " hours ago";\n}\nif (delta &lt; 172800) // 48 * 60 * 60\n{\n  return "yesterday";\n}\nif (delta &lt; 2592000) // 30 * 24 * 60 * 60\n{\n  return ts.Days + " days ago";\n}\nif (delta &lt; 31104000) // 12 * 30 * 24 * 60 * 60\n{\n  int months = Convert.ToInt32(Math.Floor((double)ts.Days / 30));\n  return months &lt;= 1 ? "one month ago" : months + " months ago";\n}\nint years = Convert.ToInt32(Math.F

In [17]:
posts_bag = both_bag.filter(lambda x: x.find("PostTypeId=\"1\"") >= 0)
posts_bag.map(xml_row_to_tuple).take(1)
#posts_bag.count().compute()

((6,
  1,
  31,
  '2008-07-31T22:08:08.620',
  261,
  16799,
  '<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to \'0\' width on <a href="http://en.wikipedia.org/wiki/Internet_Explorer_7" rel="noreferrer">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I\'m missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n',
  9,
  63550,
  'Rich B',
  '2016-03-19T06:05:48.487',
  '2018-10-16T16:54:34.953',
  'Percentage width child element in absolutely positioned parent on Internet Expl

In [18]:
## header